### Feature Extraction with an AutoEncoder

Refer to [/examples/time_series/generate_data.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/time_series/generate_data.ipynb) for the artificial time-series dataset (used in this example) generation  

$32\times 3$ matrix reduced to $8$ element vector with an autoencoder. Then, the encoder part has been merged with a simple MLP to classify *vers* of the input. Test accuracy is around $0.77$. Although the method has worked properly, [the straightforward method]([/examples/datasets_misc.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/datasets_misc.ipynb)) outperforms this feature extractor method with around $0.97$ test accuracy  

Previous example: [/examples/autoencoders/lstm.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/autoencoders/lstm.ipynb)  
Next example: [/examples/nlp/bidirectional.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/nlp/bidirectional.ipynb)

In [1]:
import sys
sys.path.insert(0, '../../') # To be able to reach 'datasets' folder
from pathlib import Path
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Sequential, Model
from keras.regularizers import l1
from keras.layers import Input, BatchNormalization, GRU, Bidirectional, Dropout, RepeatVector, Dense, TimeDistributed
from keras.callbacks import EarlyStopping
from keras.metrics import RootMeanSquaredError

In [2]:
dataset_path = Path.cwd().parent.parent / 'datasets' / 'time_series'

input = np.load(dataset_path / 'input.npy')
output = np.load(dataset_path / 'output.npy')

# Try to classify vers for within=1, inter=0
x_train, x_test, y_train, y_test = train_test_split(\
    np.concatenate((input[0], input[1])), \
    np.concatenate((np.zeros_like(output[1][0][0]), np.ones_like(output[1][0][1]))), \
    test_size=0.20, shuffle=True)

del dataset_path, input, output
def print_data(msg, data): print(f'{msg:25}: {data.shape}')
print_data('x_train', x_train)
print_data('y_train', y_train)
print_data('x_test', x_test)
print_data('y_test', y_test)

x_train                  : (49152, 32, 3)
y_train                  : (49152,)
x_test                   : (12288, 32, 3)
y_test                   : (12288,)


In [3]:
autoencoder = Sequential()
autoencoder.add(Input(x_train.shape[1:]))
autoencoder.add(BatchNormalization())
autoencoder.add(GRU(64, return_sequences=True))
autoencoder.add(Dropout(1 / 32))
autoencoder.add(GRU(8, activity_regularizer=l1(1e-4), name='encoder'))
autoencoder.add(RepeatVector(32))
autoencoder.add(Bidirectional(GRU(64, return_sequences=True)))
autoencoder.add(Dropout(1 / 64))
autoencoder.add(TimeDistributed(Dense(1)))
autoencoder.compile(optimizer='adam', loss='mse', metrics=[RootMeanSquaredError(name='rmse')])
autoencoder.summary()

Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-01-04 15:26:18.755744: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-04 15:26:18.755985: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 32, 3)            12        
 ormalization)                                                   
                                                                 
 gru (GRU)                   (None, 32, 64)            13248     
                                                                 
 dropout (Dropout)           (None, 32, 64)            0         
                                                                 
 encoder (GRU)               (None, 8)                 1776      
                                                                 
 repeat_vector (RepeatVector  (None, 32, 8)            0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 32, 128)          2

In [4]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4, restore_best_weights=True)
autoencoder.fit(x_train, x_train, epochs=256, batch_size=128, shuffle=True, validation_data=(x_test, x_test), callbacks=[early_stop])

Epoch 1/256


2023-01-04 15:26:19.409420: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-04 15:26:21.193704: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 15:26:21.540714: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 15:26:21.639820: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 15:26:21.725305: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 15:26:21.736493: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 15:26:21.912956: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114

383/384 [============================>.] - ETA: 0s - loss: 28.0412 - rmse: 5.2954

2023-01-04 15:26:40.904618: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 15:26:41.014964: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 15:26:41.079927: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 15:26:41.149656: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 15:26:41.158329: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


384/384 [==============================] - 24s 54ms/step - loss: 28.0381 - rmse: 5.2951 - val_loss: 25.4842 - val_rmse: 5.0482
Epoch 2/256
384/384 [==============================] - 20s 53ms/step - loss: 24.6337 - rmse: 4.9632 - val_loss: 23.6950 - val_rmse: 4.8677
Epoch 3/256
384/384 [==============================] - 21s 54ms/step - loss: 23.8337 - rmse: 4.8819 - val_loss: 23.5507 - val_rmse: 4.8529
Epoch 4/256
384/384 [==============================] - 20s 53ms/step - loss: 24.3988 - rmse: 4.9395 - val_loss: 23.2852 - val_rmse: 4.8254
Epoch 5/256
384/384 [==============================] - 20s 52ms/step - loss: 23.4276 - rmse: 4.8402 - val_loss: 23.0166 - val_rmse: 4.7975
Epoch 6/256
384/384 [==============================] - 20s 53ms/step - loss: 23.0471 - rmse: 4.8007 - val_loss: 22.4241 - val_rmse: 4.7354
Epoch 7/256
384/384 [==============================] - 20s 53ms/step - loss: 22.6395 - rmse: 4.7581 - val_loss: 22.3622 - val_rmse: 4.7288
Epoch 8/256
384/384 [==================

In [5]:
encoder = Model(inputs=autoencoder.inputs, outputs=autoencoder.get_layer('encoder').output)
encoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 3)]           0         
                                                                 
 batch_normalization (BatchN  (None, 32, 3)            12        
 ormalization)                                                   
                                                                 
 gru (GRU)                   (None, 32, 64)            13248     
                                                                 
 dropout (Dropout)           (None, 32, 64)            0         
                                                                 
 encoder (GRU)               (None, 8)                 1776      
                                                                 
Total params: 15,036
Trainable params: 15,030
Non-trainable params: 6
_________________________________________________________

In [6]:
model = Sequential()
model.add(encoder)
model.add(Dense(64, activation='relu'))
model.add(Dropout(1 / 32))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid', name='out'))
model.layers[0].trainable = False
model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()
print(f'{model.trainable = }')
print(f'{model.layers[0].trainable = }')
print(f'{model.layers[0].layers[0].trainable = }')
print(f'{model.layers[0].layers[1].trainable = }')
print(f'{model.layers[0].layers[-1].trainable = }')
print(f'{model.layers[1].trainable = }')
print(f'{model.layers[-1].trainable = }')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 8)                 15036     
                                                                 
 dense_1 (Dense)             (None, 64)                576       
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 out (Dense)                 (None, 1)                 65        
                                                                 
Total params: 19,837
Trainable params: 4,801
Non-trainable params: 15,036
_________________________________________________________________
model.trainable = True
model.layers[0].trainab

In [7]:
def disp_results(func, gt, pred, msg): print(f'{func.__name__} of {msg}:\n{func(gt, pred)}')
def analyse():
    y_train_pred = np.float32(model.predict(x_train, batch_size=128, verbose=0) > .5)
    y_test_pred = np.float32(model.predict(x_test, batch_size=128, verbose=0) > .5)
    disp_results(classification_report, y_train, y_train_pred, 'training data')
    disp_results(confusion_matrix, y_train, y_train_pred, 'training data')
    print()
    disp_results(classification_report, y_test, y_test_pred, 'test data')
    disp_results(confusion_matrix, y_test, y_test_pred, 'test data')

In [8]:
print('Before Training:'); analyse()

Before Training:


2023-01-04 15:39:16.279562: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 15:39:16.363283: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 15:39:16.434439: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


classification_report of training data:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00     24661
         1.0       0.50      1.00      0.67     24491

    accuracy                           0.50     49152
   macro avg       0.25      0.50      0.33     49152
weighted avg       0.25      0.50      0.33     49152

confusion_matrix of training data:
[[    0 24661]
 [    0 24491]]

classification_report of test data:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      6059
         1.0       0.51      1.00      0.67      6229

    accuracy                           0.51     12288
   macro avg       0.25      0.50      0.34     12288
weighted avg       0.26      0.51      0.34     12288

confusion_matrix of test data:
[[   0 6059]
 [   0 6229]]


/opt/miniconda3/envs/study/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/study/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/study/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/

In [9]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4, restore_best_weights=True)
model.fit(x_train, y_train, epochs=256, batch_size=128, shuffle=True, validation_data=(x_test, y_test), callbacks=[early_stop])

Epoch 1/256


2023-01-04 15:39:22.274411: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 15:39:22.429074: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


  1/384 [..............................] - ETA: 7:44 - loss: 0.6884

2023-01-04 15:39:22.502309: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


380/384 [============================>.] - ETA: 0s - loss: 0.5567

2023-01-04 15:39:27.766710: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 15:39:27.847782: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 15:39:27.916360: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


384/384 [==============================] - 8s 16ms/step - loss: 0.5567 - val_loss: 0.5159
Epoch 2/256
384/384 [==============================] - 6s 15ms/step - loss: 0.5140 - val_loss: 0.5008
Epoch 3/256
384/384 [==============================] - 6s 15ms/step - loss: 0.5070 - val_loss: 0.4970
Epoch 4/256
384/384 [==============================] - 6s 15ms/step - loss: 0.5035 - val_loss: 0.4994
Epoch 5/256
384/384 [==============================] - 6s 15ms/step - loss: 0.5008 - val_loss: 0.4899
Epoch 6/256
384/384 [==============================] - 6s 15ms/step - loss: 0.4973 - val_loss: 0.4885
Epoch 7/256
384/384 [==============================] - 6s 15ms/step - loss: 0.4951 - val_loss: 0.4850
Epoch 8/256
384/384 [==============================] - 6s 15ms/step - loss: 0.4919 - val_loss: 0.4840
Epoch 9/256
384/384 [==============================] - 6s 15ms/step - loss: 0.4922 - val_loss: 0.4839
Epoch 10/256
384/384 [==============================] - 6s 15ms/step - loss: 0.4892 - val_loss

In [10]:
print('After Training:'); analyse()

After Training:
classification_report of training data:
              precision    recall  f1-score   support

         0.0       0.77      0.78      0.78     24661
         1.0       0.78      0.77      0.77     24491

    accuracy                           0.77     49152
   macro avg       0.77      0.77      0.77     49152
weighted avg       0.77      0.77      0.77     49152

confusion_matrix of training data:
[[19285  5376]
 [ 5729 18762]]

classification_report of test data:
              precision    recall  f1-score   support

         0.0       0.77      0.78      0.77      6059
         1.0       0.78      0.77      0.78      6229

    accuracy                           0.77     12288
   macro avg       0.77      0.77      0.77     12288
weighted avg       0.77      0.77      0.77     12288

confusion_matrix of test data:
[[4732 1327]
 [1445 4784]]


Previous example: [/examples/autoencoders/lstm.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/autoencoders/lstm.ipynb)  
Next example: [/examples/nlp/bidirectional.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/nlp/bidirectional.ipynb)